In [1]:
import pandas as pd
import geopandas as gpd
import maup
from maup import smart_repair
import time
import os
import matplotlib.pyplot as plt
import networkx as nx
from gerrychain import Graph, Partition, proposals, updaters, constraints, accept, MarkovChain, Election
from gerrychain.updaters import cut_edges, Tally
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from functools import partial
import time
from gerrychain.tree import bipartition_tree

In [2]:
ct_file = gpd.read_file("./WI_DATA/WI.shp") # reads a file from ct directory
ct_graph = Graph.from_geodataframe(ct_file) # creates a graph from a geodataframe
print(ct_graph.nodes()[0]) # Prints out the attributes of each node in the graph
tot_pop = sum([ct_graph.nodes()[v]['TOTPOP'] for v in ct_graph.nodes() if not pd.isna(ct_graph.nodes()[v]['TOTPOP'])]) # gets the total population for ideal population calculation if the values are numbers
print(tot_pop)

/Users/williamstout/miniconda3/envs/gerry/lib/python3.11/site-packages/gerrychain/graph/graph.py:406: UserWarning: Found islands (degree-0 nodes). Indices of islands: {1220, 5148}
  warnings.warn(


{'boundary_node': False, 'area': 664968.208542657, 'CNTY_FIPS': '55035', 'CNTY_NAME': 'EAU CLAIRE', 'MCD_FIPS': '5503522300', 'MCD_NAME': 'EAU CLAIRE', 'CTV': 'C', 'WARDID': '0060', 'LABEL': 'EAU CLAIRE - C 0060', 'PERSONS': 12, 'RV': 0, 'G20PRED': 0, 'G20PRER': 0, 'TOTPOP': 4.0, 'HISP': 0.0, 'NH_WHITE': 0.0, 'NH_BLACK': 1.0, 'NH_AMIN': 0.0, 'NH_ASIAN': 0.0, 'NH_NHPI': 0.0, 'NH_OTHER': 0.0, 'NH_2MORE': 3.0, 'CD': '3', 'geometry': <POLYGON ((139292.952 4974890.501, 139269.277 4974487.775, 139622.622 497446...>}
5893718.0


In [3]:
ct_partition = Partition( # creates a partition for the IL graph
    ct_graph, # dual graph
    assignment = "CD", # initial districting plan
    updaters={
    "our cut edges": cut_edges, # number of cut edges in a graph
    "district population": Tally("TOTPOP", alias = "district population"), # across districts, total population 
    "hispanic population": Tally("HISP", alias = "hispanic population"), # across districts, hispanic population 
    "democratic votes": Tally("G20PRED", alias = "democratic votes"), # across districts, number of democratic votes
    "republican votes": Tally("G20PRER", alias = "republican votes"), # across districts, number of republican votes
})
print(ct_partition["district population"]) # Prints the population for each district
print(ct_partition["democratic votes"]) # prints the number of democratic votes for each district
print(ct_partition["republican votes"]) # prints the number of republican votes for each district
ideal_pop = tot_pop/8 # calculates the ideal population 
pop_tolerance = 0.05 # population tolerance for the proposal
print(ideal_pop)

{'3': 736959.0, '2': 736822.0, '6': 736545.0, '4': 736395.0, '1': 736286.0, '8': 736833.0, '7': 736572.0, '5': 737306.0}
{'3': 185134, '2': 310915, '6': 171907, '4': 260053, '1': 189936, '8': 170943, '7': 165821, '5': 176157}
{'3': 203656, '2': 125964, '6': 237024, '4': 78197, '1': 197794, '8': 234798, '7': 250139, '5': 282612}
736714.75


In [4]:
rw_proposal = partial(recom, ## how you choose a next districting plan
                      pop_col = "TOTPOP", ## What data describes population? 
                      pop_target = ideal_pop, ## What the target/ideal population is for each district 
                                              ## (we calculated ideal pop above)
                      epsilon = pop_tolerance,  ## how far from ideal population you can deviate
                                              ## (we set pop_tolerance above)
                      node_repeats = 100, ## number of times to repeat bipartition.  Can increase if you get a BipartitionWarning
                    #   allow_pair_reselection=True,
                        method = partial(
                        bipartition_tree,
                        max_attempts=100,
                        allow_pair_reselection=True  # <-- This is the only change
    )
                      )
population_constraint = constraints.within_percent_of_ideal_population(
    ct_partition, 
    pop_tolerance, 
    pop_key="district population")

In [ ]:
# found this code in gerrychain_intro_complete-2.ipynb
start_time = time.time()
random_walk_10 = MarkovChain( # random walk for 10000 steps
    proposal = rw_proposal, 
    constraints = [population_constraint], # Could add additional constraints here
    accept = always_accept, # Accept every proposed plan that meets the population constraints
    initial_state = ct_partition, 
    total_steps = 100)

cutedge_ensemble_10 = [] ## Cut edge ensemble for Markov Chain with 10000 steps
hisp_maj_ensemble_10 = [] ## Hisp majority ensemble for Markov Chain with 10000 steps
dem_maj_ensemble_10 = [] ## Dem majority ensemble for Markov Chain with 10000 steps

for j, part in enumerate(random_walk_10):
    # Add cutedges to cutedges ensemble
    cutedge_ensemble_10.append(len(part["our cut edges"]))
    # Calculate number of hisp-majority and dem-majority districts 
    # Add to ensemble
    num_maj_hisp = 0
    num_maj_dem = 0
    
    for i in range(1,9):
        district = str(i)
        b_perc = part["hispanic population"][district] / part["district population"][district] # calculation for hispanic population 
        dem_perc = part["democratic votes"][district] / (part["democratic votes"][district] + part["republican votes"][district]) # calculation for democratic vote share
        if dem_perc >= 0.5: # if majority democratic
            num_maj_dem = num_maj_dem + 1
        if b_perc >= 0.5: # if majority hispanic
            num_maj_hisp = num_maj_hisp + 1
    hisp_maj_ensemble_10.append(num_maj_hisp)
    dem_maj_ensemble_10.append(num_maj_dem)
end_time = time.time()

print("The time of execution of above program is :",
      (end_time-start_time)/60, "mins")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
The time of execution of above program is : 0.6463001330693563 mins


In [7]:
print(hisp_maj_ensemble_10)
print(cutedge_ensemble_10)
print(dem_maj_ensemble_10)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[672, 697, 684, 619, 631, 625, 664, 668, 684, 675, 679, 671, 657, 667, 687, 650, 635, 669, 682, 731, 706, 693, 716, 701, 713, 737, 747, 756, 708, 743, 682, 739, 774, 780, 786, 789, 831, 872, 849, 908, 906, 940, 949, 936, 917, 928, 938, 926, 834, 827, 810, 778, 766, 703, 738, 696, 741, 723, 752, 802, 768, 756, 718, 727, 703, 740, 754, 729, 726, 721, 757, 762, 761, 723, 771, 778, 774, 776, 710, 706, 717, 739, 734, 653, 636, 611, 603, 626, 610, 627, 638, 625, 615, 630, 648, 689, 681, 673, 677, 654]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3,